In [4]:
import PyQt6.QtCore
import os
os.environ["QT_API"] = "pyqt5"


# Use qt backend for matplotlab to use interactive mne plots
%matplotlib qt

ModuleNotFoundError: No module named 'PyQt6'

In [14]:
import mne 
import analysis.processing
import matplotlib.pyplot as plt
import pandas as pd
import csv 
import os
from config import Config
configObj = Config()
configss = configObj.getConfigSnapshot()

mne.set_log_level(verbose='WARNING', return_old_level=False, add_frames=None)

In [2]:
analysis.processing.segmentData([1,2,4], preprocess = True)

c:\Users\erang\Desktop\Reading_task\src\analysis\processing.py:54: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
c:\Users\erang\Desktop\Reading_task\src\analysis\processing.py:290: RuntimeWarning: This filename (c:\Users\erang\Desktop\Reading_task\ov_experiment\scenarios\data\Pilot\P1\D0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  partipant_data_path =  'Pilot'+ '/'+ participant_number + '/' + block_number +'.fif'
c:\Users\erang\Desktop\Reading_task\src\analysis\processing.py:302: RuntimeWarning: This filename (c:\Users\erang\Desktop\Reading_task\ov_experiment\scenarios\data\Pilot\P1\ND0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _e

In [15]:
for pnum in (1,2,4):
    for b_cnt in range (0, 2):
        block_nums = [ x+ (str(b_cnt)) for x in ['D', 'ND']  ]
        for block_num in block_nums:
            raw = analysis.processing.loadSegmentedData(pnum, block_num)
            epochs = analysis.processing.epochContinuousData(raw)
            
            participant_number = 'P' + str(pnum)
            partipant_data_path =  'Pilot'+ '/'+ participant_number + '/' + block_num +'-epo.fif'
            path = os.path.join(configss['root'], configss['data_dir'] , partipant_data_path ) 

            epochs.save(path, overwrite= True)
            
        

c:\Users\erang\Desktop\Reading_task\src\analysis\processing.py:26: RuntimeWarning: This filename (c:\Users\erang\Desktop\Reading_task\ov_experiment/scenarios/data\Pilot/P1/D0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw  = mne.io.read_raw_fif(path)
c:\Users\erang\Desktop\Reading_task\src\analysis\processing.py:26: RuntimeWarning: This filename (c:\Users\erang\Desktop\Reading_task\ov_experiment/scenarios/data\Pilot/P1/ND0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw  = mne.io.read_raw_fif(path)
c:\Users\erang\Desktop\Reading_task\src\analysis\processing.py:26: RuntimeWarning: This filename (c:\Users\erang\De

In [2]:
# read group evokeds

oddball_list = []
control_list = []

for pnum in (11, 12, 13, 21, 22, 23, 41, 42, 43, 51, 52, 53, 61, 62, \
             63, 71, 72, 73, 81, 82, 83, 91, 92, 93, 101, 102, 103):
    raw = analysis.processing.loadData(pnum)

    # preprocessig 
    raw = analysis.processing.preprocessing(raw)
    epochs, event_dict = analysis.processing.eventEpocshByBlocks(raw)

    for bnum in (0, 1, 2):
        block = "b" + str(bnum)

        oddball = epochs[f"oddball & block == '{block}'"].average()
        control = epochs[f"control & block == '{block}'"].average()
        oddball_list.append(oddball)
        control_list.append(control)



title= "ERP"
roi =['7Z']

evokeds = dict(
    oddball=  oddball_list,
    control= control_list,
)

c:\Users\erang\Desktop\Auditory-oddball\src\analysis\processing.py:46: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
c:\Users\erang\Desktop\Auditory-oddball\src\analysis\processing.py:46: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
c:\Users\erang\Desktop\Auditory-oddball\src\analysis\processing.py:46: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
c:\Users\erang\Desktop\Auditory-oddball\src\analysis\processing.py:46: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filtered.set_montage(montage)
c:\Users\erang\Desktop\Auditory-oddball\src\analysis\processing.py:46: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw_filte

In [5]:
analysis.processing.getERPMontage(evokeds)

In [6]:
#plot evokeds at 0.95 CI
analysis.processing.getERP(evokeds, roi = ['7Z'], ci = 0.95, invert = True)

In [9]:
#plot evokeds at 0.9 CI
analysis.processing.getERP(evokeds, roi = ['7Z'], ci = 0.9, invert = True)

In [15]:
#front

roi = ['1L', '1R','1Z','1LC', '1LB', '2L', '2R', '1RB', '1RC']
ci = 0.95
invert = False

fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks= roi, ci = ci , 
                                styles = {"oddball": {"color" :'red'}, 
                                        "control":{"color": 'blue'}}, 
                                    show =False, axes = ax, combine ='mean' )
if(invert): 
    ax.invert_yaxis()
    
ax.set_ylabel("V")

plt.show()

In [14]:
#Occipital

roi = ['10L', '9Z','10R', '9L', '8Z', '9R']
ci = 0.95
invert = False

fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks= roi, ci = ci , 
                                styles = {"oddball": {"color" :'red'}, 
                                        "control":{"color": 'blue'}}, 
                                    show =False, axes = ax, combine ='mean' )
if(invert): 
    ax.invert_yaxis()
    
ax.set_ylabel("V")

plt.show()

In [12]:
#Left

roi = ['3LB', '3LC','2LB', '1LA', '2LA', '2LC']
ci = 0.95
invert = False

fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks= roi, ci = ci , 
                                styles = {"oddball": {"color" :'red'}, 
                                        "control":{"color": 'blue'}}, 
                                    show =False, axes = ax, combine ='mean' )
if(invert): 
    ax.invert_yaxis()
    
ax.set_ylabel("V")

plt.show()

In [13]:
#Right

roi = ['3RB', '3RC','2RB', '1RA', '2RA', '2RC']
ci = 0.95
invert = False

fig, ax = plt.subplots()
mne.viz.plot_compare_evokeds(evokeds, picks= roi, ci = ci , 
                                styles = {"oddball": {"color" :'red'}, 
                                        "control":{"color": 'blue'}}, 
                                    show =False, axes = ax, combine ='mean' )
if(invert): 
    ax.invert_yaxis()
    
ax.set_ylabel("V")

plt.show()